In [2]:
import pandas as pd
import numpy as np

tags = pd.read_csv('tags.csv')
rating = pd.read_csv('ratings.csv')
to_read = pd.read_csv('to_read.csv')
book = pd.read_csv('book_no_null.csv')
book_tags = pd.read_csv('book_tags.csv')

## AssociationRules

In [3]:
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [122]:
to_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912705 entries, 0 to 912704
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  912705 non-null  int64
 1   book_id  912705 non-null  int64
dtypes: int64(2)
memory usage: 13.9 MB


In [123]:
to_read.isna().sum()

user_id    0
book_id    0
dtype: int64

### creating one hot from to_read dataset

In [4]:
read_1_hot = pd.DataFrame(0,columns=[i for i in range(1,10001)],index=[i for i in range(1,53425)])

In [6]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        read_1_hot.loc[_uid_, _bid_] = 1

In [8]:
read_1_hot

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
read_1_hot.to_csv('to_read_1_hot.csv')

In [9]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [10]:
frq_items_apriori = fpgrowth(read_1_hot, min_support = 0.0025, use_colnames = True)
frq_items_apriori.sort_values(by=['support'], inplace=True)

In [11]:
frq_items_apriori.shape

(2181, 2)

In [12]:
frq_items_apriori

,support,itemsets
1613,0.002508,(218)
1508,0.002508,(419)
1649,0.002508,(5223)
1840,0.002508,"(35, 119)"
1948,0.002508,"(66, 103)"
...,...,...
9,0.033075,(11)
8,0.033917,(13)
7,0.034441,(113)
125,0.036819,(143)


In [13]:
rules = association_rules(frq_items_apriori, metric ="lift", min_threshold = 1)

In [14]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(35),(119),0.029500,0.028059,0.002508,0.085025,3.030284,0.001681,1.062261
1,(119),(35),0.028059,0.029500,0.002508,0.089393,3.030284,0.001681,1.065773
2,(66),(103),0.021676,0.019916,0.002508,0.115717,5.810199,0.002077,1.108337
3,(103),(66),0.019916,0.021676,0.002508,0.125940,5.810199,0.002077,1.119287
4,(94),(95),0.027329,0.022612,0.002508,0.091781,4.059022,0.001890,1.076159
...,...,...,...,...,...,...,...,...,...
1591,(135),(165),0.010613,0.011923,0.006608,0.622575,52.214198,0.006481,2.617941
1592,(17),(20),0.011062,0.012766,0.006776,0.612521,47.981422,0.006635,2.547840
1593,(20),(17),0.012766,0.011062,0.006776,0.530792,47.981422,0.006635,2.107673
1594,(188),(165),0.015592,0.011923,0.007731,0.495798,41.581679,0.007545,1.959685


### creating to_read_authors dataset using to_read dataset

In [120]:
to_read

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380
...,...,...
912700,39374,1049
912701,10492,5180
912702,21879,4827
912703,21879,6642


In [124]:
import re
unique_authors = set()
for i in book['authors'].unique():
    res = re.split(',', i)
    for j in res:
        unique_authors.add(j.lstrip())

In [125]:
len(unique_authors)

5841

In [132]:
authors_1_hot = pd.DataFrame(0,columns=[i for i in unique_authors],index=[i for i in range(0,53425)])

In [ ]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_1_hot.loc[_uid_, _au_.lstrip()] = 1

In [135]:
authors_1_hot

,Gert Westphal,Hisaya Nakajo,C.S. Lewis,David Foster Wallace,Cynthia Marshall,Sherwood Anderson,Karin Slaughter,Larissa Volokhonsky,Hans Christian Andersen,Alex Ross,...,Gareth Stedman Jones,Richard Condon,Alisa Valdes-Rodriguez,Ivan Goncharov,Frank Pittarese,William Benham,Brent Curtis,Gene Zion,Edward Bloor,Chris Ryall
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
authors_1_hot.to_csv('authors_1_hot.csv')

In [136]:
frq_items_apriori = fpgrowth(authors_1_hot, min_support = 0.01, use_colnames = True)
frq_items_apriori.sort_values(by=['support'], inplace=True)

In [137]:
frq_items_apriori.shape

(651, 2)

In [138]:
frq_items_apriori

,support,itemsets
418,0.010033,(Brandon Mull)
606,0.010033,"(Neil Gaiman, Orson Scott Card)"
341,0.010033,(Ruth Ware)
444,0.010033,"(Jeffrey Eugenides, Gabriel García Márquez)"
372,0.010070,(John E. Woods)
...,...,...
366,0.067309,(Cassandra Clare)
7,0.069949,(Khaled Hosseini)
165,0.095405,(J.K. Rowling)
146,0.111072,(Stephen King)


In [139]:
rules = association_rules(frq_items_apriori, metric ="lift", min_threshold = 1)

In [140]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Neil Gaiman),(Orson Scott Card),0.119326,0.035489,0.010033,0.084078,2.369140,0.005798,1.053050
1,(Orson Scott Card),(Neil Gaiman),0.035489,0.119326,0.010033,0.282700,2.369140,0.005798,1.227763
2,(Jeffrey Eugenides),(Gabriel García Márquez),0.060103,0.047431,0.010033,0.166926,3.519350,0.007182,1.143439
3,(Gabriel García Márquez),(Jeffrey Eugenides),0.047431,0.060103,0.010033,0.211523,3.519350,0.007182,1.192042
4,(Stephen King),(J.R.R. Tolkien),0.111072,0.045952,0.010070,0.090664,1.973003,0.004966,1.049170
...,...,...,...,...,...,...,...,...,...
697,(Louise Maude),(Aylmer Maude),0.035583,0.036668,0.035583,1.000000,27.271567,0.034278,inf
698,(Aylmer Maude),(Leo Tolstoy),0.036668,0.036874,0.036668,1.000000,27.119289,0.035316,inf
699,(Leo Tolstoy),(Aylmer Maude),0.036874,0.036668,0.036668,0.994416,27.119289,0.035316,172.523963
700,(Reg Keeland),(Stieg Larsson),0.037342,0.043556,0.037342,1.000000,22.958745,0.035716,inf
